<a href="https://colab.research.google.com/github/maryamt95/project-files/blob/main/north_climate_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# mounting drive for data access

In [ ]:
from google.colab import drive  
drive._mount('/content/drive') 

# installing libraries


In [ ]:
pip install cdsapi

In [1]:
import cdsapi
import netCDF4 as nc
from netCDF4 import num2date
from netCDF4 import Dataset
import numpy as np
import os
import pandas as pd

In [ ]:
import xarray as xr
import os



# Read Data from Drive 

In [2]:
north_data=pd.read_csv('/content/drive/MyDrive/data/ERA5_monthly_north.csv')

In [7]:
north_data.dtypes

expver         int64
latitude     float64
longitude    float64
time          object
t2m          float64
siconc       float64
asn          float64
smlt         float64
sp           float64
tco3         float64
dtype: object

In [ ]:
len(north_data)

141177600

In [ ]:
north_data.count()

expver       141177600
latitude     141177600
longitude    141177600
time         141177600
t2m           70588800
siconc        52533444
asn           70588800
smlt          70588800
sp            70588800
tco3          70588800
dtype: int64

# Rename

In [ ]:
north_data__renamed=north_data.rename(columns={'t2m':'temperature_2m','siconic':'sea_ice_area_fraction','asn':'Snow_albedo','smlt':'snow_melt','sp':'surface_pressure','tco3':
'Total_column_ozone'})

In [ ]:
north_d=north_data.drop(['expver','latitude','longitude'],axis=1)

In [ ]:
north_avg=north_d.groupby('time').mean() #keeps crushing 

In [ ]:
north_data.head(2)

,expver,latitude,longitude,time,t2m,siconc,asn,smlt,sp,tco3
0,1,90.0,-180.0,1979-01-01,244.70773,0.976897,0.879997,0.0,102661.984,0.007461
1,1,90.0,-180.0,1979-02-01,241.44653,0.987060,0.880002,0.0,101824.310,0.009453


In [ ]:
#separating columns 

In [ ]:
north_data__renamed=north_d.rename(columns={'t2m':'temperature_2m','siconic':'sea_ice_area_fraction','asn':'Snow_albedo','smlt':'snow_melt','sp':'surface_pressure','tco3':
'Total_column_ozone'})

#Calculating averages 

In [3]:
north_temp=north_data[['time','t2m']]
north_temp_avg=north_temp.groupby('time').mean()

In [20]:
north_temp_avg.to_csv('temp.csv') 

In [8]:
# re- indexing to eliminate the index error from sea ice 
#columns=['expver','latitude','longitude','time','t2m','siconc','asn','smlt','sp','tco3']
#north_data=north_data.reindex(columns=columns)

In [ ]:
#north_seaice=north_data[['time','siconic']]
#north_seaice_avg=north_seaice.groupby('time').mean()

In [4]:
north_snow_albedo=north_data[['time','asn']]
north_snow_albedo_avg=north_snow_albedo.groupby('time').mean()

In [ ]:
north_snow_albedo_avg.to_csv('snoe_albedo.csv')

In [5]:
north_snow_melt=north_data[['time','smlt']]
north_snow_melt_avg=north_snow_melt.groupby('time').mean()

In [ ]:
north_snow_melt_avg.to_csv('snow_melt.csv')

In [6]:
north_spressure=north_data[['time','sp']]
north_spressure_avg=north_spressure.groupby('time').mean()

In [ ]:
north_spressure_avg.to_csv('spressure.csv')

In [ ]:
north_ozon=north_data[['time','tco3']]
north_ozon_avg=north_ozon.groupby('time').mean()

In [ ]:
north_ozon_avg.to_csv('ozon.csv')

# Merging

In [7]:
north_merged=pd.merge(north_temp_avg,north_snow_albedo_avg,on="time")

In [8]:
north_merged_1=pd.merge(north_merged,north_snow_melt_avg,on="time")

In [9]:
north_merged_2=pd.merge(north_merged_1,north_spressure_avg,on="time")

In [10]:
north_data_r=north_merged_2.rename(columns={'t2m':'temperature_2m','asn':'Snow_albedo','smlt':'snow_melt','sp':'surface_pressure'})

In [11]:
north_data_r.head()

,temperature_2m,Snow_albedo,snow_melt,surface_pressure
time,,,,
1979-01-01,248.048989,0.842807,2.149000e-07,99971.627492
1979-02-01,243.439591,0.836100,7.621431e-07,99641.237045
1979-03-01,249.055998,0.834508,8.765569e-07,100240.042241
1979-04-01,254.599298,0.838258,3.177189e-05,100369.270964
1979-05-01,265.495457,0.842589,3.034634e-04,99839.279702


In [19]:
north_data_r.count()

temperature_2m      516
Snow_albedo         516
snow_melt           516
surface_pressure    516
dtype: int64

In [12]:
ozon=pd.read_csv('/content/ozon.csv')

In [15]:
north_data_avg=pd.merge(north_data_r,ozon,on="time")
north_data_avg=north_data_avg.rename(columns={'tco3':'Total_column_ozone'})

In [16]:
north_data_avg.head()

,time,temperature_2m,Snow_albedo,snow_melt,surface_pressure,Total_column_ozone
0,1979-01-01,248.048989,0.842807,2.149000e-07,99971.627492,0.007569
1,1979-02-01,243.439591,0.836100,7.621431e-07,99641.237045,0.009261
2,1979-03-01,249.055998,0.834508,8.765569e-07,100240.042241,0.010040
3,1979-04-01,254.599298,0.838258,3.177189e-05,100369.270964,0.009609
4,1979-05-01,265.495457,0.842589,3.034634e-04,99839.279702,0.008514


In [21]:
north_data_avg.to_csv('north_avg.csv')
!cp north_avg.csv "drive/My Drive/data/"


In [18]:
north_data_avg.count()

time                  516
temperature_2m        516
Snow_albedo           516
snow_melt             516
surface_pressure      516
Total_column_ozone    516
dtype: int64